In [1]:
%pip install numpy pandas backtesting joblib

   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ----------- ---------------------------- 2.1/7.0 MB 11.3 MB/s eta 0:00:01
   -------------------------- ------------- 4.7/7.0 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 11.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

import joblib

import numpy as np
from backtesting import Backtest, Strategy
from backtesting.test import BTCUSD




c:\Users\JTYAP\AppData\Local\Programs\Python\Python313\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [ ]:
hmmodel = joblib.load("../models/hmm.pkl")

training_df = pd.read_csv("../datasets/BTCUSDT_train_features.csv")


volume_mean = training_df['volume'].mean()
volume_std = training_df['volume'].std()

volume_mean = training_df['volume'].mean()
volume_std = training_df['volume'].std()

rsi_mean = training_df['rsi'].mean()
rsi_std = training_df['rsi'].std()

macd_mean = training_df['macd'].mean()
macd_std = training_df['macd'].std()

ema12_mean = training_df['ema_12'].mean()
ema12_std = training_df['ema_12'].std()

ema26_mean = training_df['ema_26'].mean()
ema26_std = training_df['ema_26'].std()

sma20_mean = training_df['sma_20'].mean()
sma20_std = training_df['sma_20'].std()

volatility_mean = training_df['volatility'].mean()
volatility_std = training_df['volatility'].std()

def HMM_predictor(today_close, yesterday_close, today_vol, today_rsi, today_macd, today_ema12, today_ema26, today_sma20, today_volatility):

    # Calculate features
    log_return = np.log(today_close / yesterday_close)
    volume_norm = (today_vol - volume_mean) / volume_std
    rsi_norm = (today_rsi - rsi_mean) / rsi_std
    macd_norm = (today_macd - macd_mean) / macd_std
    ema12_norm = (today_ema12 - ema12_mean) / ema12_std
    ema26_norm = (today_ema26 - ema26_mean) / ema26_std
    sma20_norm = (today_sma20 - sma20_mean) / sma20_std
    volatility_norm = (today_volatility - volatility_mean) / volatility_std

    # Format as 2D array
    new_obs = np.array([[log_return, volume_norm, rsi_norm, macd_norm, ema12_norm, ema26_norm, sma20_norm, volatility_norm]])

    # Predict hidden state
    state_today = hmmodel.predict(new_obs)

    return state_today[0]

In [4]:

# class HMMStrategy(Strategy):
#     def init(self):
#         self.data_close = self.data.Close
#         self.data_volume = self.data.Volume

#     def next(self):
#         print("Current position: ", self.position)
#         if len(self.data_close) < 2:
#             return  # Not enough data
        

#         # Get today's and yesterday's close and today's volume
#         today_close = self.data_close[-1]
#         yesterday_close = self.data_close[-2]
#         today_vol = self.data_volume[-1]

#         # Predict state using HMM
#         predicted_state = HMM_predictor(today_close, yesterday_close, today_vol)
#         print(f"Predicted state: {predicted_state}")

#         # Define your action based on predicted state (assume 2 = bullish, 0 = bearish, 1 = neutral)
#         if predicted_state == 1:
#             # Bullish -> enter long
#             if not self.position:
#                 self.buy()
#                 print("Buying")
#         elif predicted_state in [0,2]:
#             # Bearish -> close long if exists
#             if self.position:
#                 self.position.close()
#                 print("Closing position")


In [5]:
class HMMStrategy(Strategy):
    def init(self):
        self.holding_period = 0

    def next(self):
        today_close = self.data.Close[-1]
        yesterday_close = self.data.Close[-2]
        today_vol = self.data.Volume[-1]

        predicted_state = HMM_predictor(today_close, yesterday_close, today_vol)
        print(f"Predicted state: {predicted_state}")
        
        print("Self holding period: ", self.holding_period)
        if self.position:
            self.holding_period += 1
        else:
            self.holding_period = 0

      
        if self.holding_period >= 30 or predicted_state == 2:
            if self.position:
                self.position.close()
                self.holding_period = 0
                print("Closing position")
        elif predicted_state == 1:
            if not self.position:
                self.buy()
                print("Buying")


In [6]:
backtest_df = pd.read_csv("../datasets/BTC-USD_1h_Backtest_2021_to_2023.csv")

backtest_df.rename(columns={'timestamp': 'Date', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

print(backtest_df)


bt = Backtest(backtest_df, HMMStrategy, cash=1000000, commission=0.0006)
stats = bt.run()
# bt.plot()

# print(BTCUSD)

# bt = Backtest(BTCUSD, HMMStrategy, cash=100000, commission=0.0006)
# stats = bt.run()
# bt.plot()


                      Date      Open      High       Low     Close  \
0      2020-12-31 16:00:00  28820.87  28854.99  28308.28  28399.99   
1      2020-12-31 17:00:00  28399.98  28777.00  28379.00  28595.26   
2      2020-12-31 18:00:00  28594.03  28947.00  28500.00  28917.86   
3      2020-12-31 19:00:00  28920.73  29055.00  28795.46  28949.67   
4      2020-12-31 20:00:00  28949.64  29197.99  28921.01  29185.05   
...                    ...       ...       ...       ...       ...   
26991  2024-01-30 11:00:00  43472.51  43558.98  43407.26  43526.96   
26992  2024-01-30 12:00:00  43526.95  43872.99  43253.55  43394.50   
26993  2024-01-30 13:00:00  43388.55  43480.75  43144.99  43291.84   
26994  2024-01-30 14:00:00  43290.68  43507.24  43115.47  43479.72   
26995  2024-01-30 15:00:00  43479.72  43630.93  43293.12  43379.53   

            Volume  
0      1120.673910  
1      1360.142505  
2      1380.669440  
3      1558.176088  
4      1306.598512  
...            ...  
26991   321.

C:\Users\JTYAP\AppData\Local\Temp\ipykernel_11604\3656171639.py:8: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(backtest_df, HMMStrategy, cash=1000000, commission=0.0006)


ValueError: operands could not be broadcast together with shapes (1,2) (8,) 

In [ ]:
print (stats)

Start                                     0.0
End                                   26995.0
Duration                              26995.0
Exposure Time [%]                    99.81849
Equity Final [$]                 379462.89169
Equity Peak [$]                 1905541.37112
Commissions [$]                  692902.97327
Return [%]                          -62.05371
Buy & Hold Return [%]                52.74488
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Alpha [%]                          -109.19783
Beta                                  0.89381
Max. Drawdown [%]                     -89.557
Avg. Drawdown [%]                    -5.29069
Max. Drawdown Duration                24509.0
Avg. Drawdown Duration              612.56818
# Trades                                920.0
Win Rate [%]                      